In [ ]:
# Các thư viện phụ thuộc cần có
%pip install numpy>=2.0.0
%pip install scipy>=1.14.0
%pip install thinc tsfresh
%pip install gensim



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import os

# Thiết lập logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Tải NLTK data
nltk.download('punkt_tab')

In [ ]:
# Tạo dữ liệu mẫu
sample_texts = [
    "This is a sample document about machine learning.",
    "Deep learning is a subset of machine learning.",
    "Natural language processing is fascinating.",
    "Python is a popular programming language for AI.",
    "Data science combines statistics and programming.",
    "Neural networks are inspired by biological brains.",
    "Machine learning algorithms learn from data.",
    "Deep learning models require large datasets.",
    "Artificial intelligence is changing the world.",
    "Computer vision helps machines understand images."
]

# Tạo DataFrame và lưu thành CSV
df = pd.DataFrame({'text': sample_texts})
df.to_csv('sample_data.csv', index=False)
print("Đã tạo file sample_data.csv")

In [ ]:
from gensim.downloader import load
import os

def download_word2vec():
    pretrained_path = 'word2vec_pretrained.bin'
    
    # Kiểm tra xem file đã tồn tại chưa
    if not os.path.exists(pretrained_path):
        print("Đang tải pretrained Word2Vec model...")
        word2vec_model = load('word2vec-google-news-300')
        print("Đang lưu model...")
        word2vec_model.save_word2vec_format(pretrained_path, binary=True)
        print(f"Đã lưu model tại: {pretrained_path}")
    else:
        print(f"Pretrained model đã tồn tại tại: {pretrained_path}")
    
    return pretrained_path

# Tải và lưu model
PRETRAINED_PATH = download_word2vec()

In [4]:
class TextDataset:
    def __init__(self, texts, max_length=1000):
        """
        Khởi tạo dataset từ danh sách các văn bản
        texts: list các văn bản
        max_length: độ dài tối đa của mỗi văn bản
        """
        self.texts = texts
        self.max_length = max_length
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0
        self.doc_vectors = []
        
        # Xây dựng từ điển
        self._build_vocab()
        
    def _build_vocab(self):
        """Xây dựng từ điển từ tất cả các văn bản"""
        word_freq = defaultdict(int)
        
        # Đếm tần suất từ
        for text in self.texts:
            words = word_tokenize(text.lower())
            for word in words:
                word_freq[word] += 1
        
        # Tạo từ điển với các từ xuất hiện ít nhất 2 lần
        words = [word for word, freq in word_freq.items() if freq >= 2]
        for idx, word in enumerate(words):
            self.word2idx[word] = idx
            self.idx2word[idx] = word
        
        self.vocab_size = len(self.word2idx)
        
    def tokenize_text(self, text):
        """Chuyển văn bản thành chuỗi các index"""
        words = word_tokenize(text.lower())
        return [self.word2idx[word] for word in words 
                if word in self.word2idx][:self.max_length]

In [5]:
class Doc2VecDataset(Dataset):
    def __init__(self, text_dataset, window_size=5):
        self.text_dataset = text_dataset
        self.window_size = window_size
        self.data = self._prepare_training_data()
        
    def _prepare_training_data(self):
        """Chuẩn bị dữ liệu huấn luyện với cửa sổ ngữ cảnh"""
        training_data = []
        
        for doc_id, text in enumerate(self.text_dataset.texts):
            word_indices = self.text_dataset.tokenize_text(text)
            
            for target_pos, target_idx in enumerate(word_indices):
                # Lấy các từ trong cửa sổ ngữ cảnh
                context_indices = []
                for i in range(-self.window_size, self.window_size + 1):
                    context_pos = target_pos + i
                    if i != 0 and 0 <= context_pos < len(word_indices):
                        context_indices.append(word_indices[context_pos])
                
                for context_idx in context_indices:
                    training_data.append((target_idx, doc_id, context_idx))
                    
        return training_data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        target_idx, doc_id, context_idx = self.data[idx]
        return (
            torch.tensor(target_idx, dtype=torch.long),
            torch.tensor(doc_id, dtype=torch.long),
            torch.tensor(context_idx, dtype=torch.long)
        )

In [6]:
class Doc2VecWithPretrained(nn.Module):
    def __init__(self, pretrained_path, vocab_size, doc_count, vector_size=300):
        super(Doc2VecWithPretrained, self).__init__()
        
        # Tải pretrained word embeddings
        self.word_vectors = KeyedVectors.load_word2vec_format(pretrained_path, binary=True)
        weights = torch.FloatTensor(self.word_vectors.vectors)
        
        # Khởi tạo embedding layers
        self.word_embeddings = nn.Embedding.from_pretrained(weights, freeze=False)
        self.doc_embeddings = nn.Embedding(doc_count, vector_size)
        
        # Khởi tạo doc embeddings
        nn.init.uniform_(self.doc_embeddings.weight, -0.1, 0.1)
        
        # Layers
        self.combine_layer = nn.Linear(vector_size * 2, vector_size)
        self.output_layer = nn.Linear(vector_size, vocab_size)
        
        self.dropout = nn.Dropout(0.2)
        self.activation = nn.ReLU()

    def forward(self, word_ids, doc_ids):
        word_embeds = self.word_embeddings(word_ids)
        doc_embeds = self.doc_embeddings(doc_ids)
        
        # Kết hợp embeddings
        combined = torch.cat((word_embeds, doc_embeds), dim=1)
        hidden = self.activation(self.combine_layer(combined))
        hidden = self.dropout(hidden)
        output = self.output_layer(hidden)
        
        return output

In [7]:
def train_doc2vec(model, train_loader, val_loader, epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer với learning rates khác nhau
    word_embed_params = {'params': model.word_embeddings.parameters(), 'lr': learning_rate * 0.1}
    other_params = {'params': [p for n, p in model.named_parameters() 
                             if 'word_embeddings' not in n], 'lr': learning_rate}
    
    optimizer = optim.Adam([word_embed_params, other_params])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    
    best_val_loss = float('inf')
    for epoch in range(epochs):
        # Training
        model.train()
        total_loss = 0
        for batch_idx, (words, docs, targets) in enumerate(train_loader):
            words, docs, targets = words.to(device), docs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(words, docs)
            loss = criterion(outputs, targets)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()
            
            total_loss += loss.item()
            
            if batch_idx % 100 == 0:
                logging.info(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')
        
        avg_train_loss = total_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for words, docs, targets in val_loader:
                words, docs, targets = words.to(device), docs.to(device), targets.to(device)
                outputs = model(words, docs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        scheduler.step(avg_val_loss)
        
        logging.info(f'Epoch {epoch}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}')
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_doc2vec_model.pth')

In [8]:
def load_and_preprocess_data(data_path):
    """Load and preprocess text data"""
    # Đọc dữ liệu (giả sử là file CSV với cột 'text')
    df = pd.read_csv(data_path)
    texts = df['text'].tolist()
    
    # Chia train/val
    train_texts, val_texts = train_test_split(texts, test_size=0.2, random_state=42)
    
    return train_texts, val_texts

def get_document_vector(model, text, text_dataset):
    """Lấy vector biểu diễn cho văn bản mới"""
    model.eval()
    with torch.no_grad():
        # Tokenize văn bản
        word_indices = text_dataset.tokenize_text(text)
        if not word_indices:
            return None
        
        # Tính trung bình các word vectors
        word_vectors = model.word_embeddings(torch.tensor(word_indices))
        doc_vector = word_vectors.mean(dim=0)
        
        return doc_vector.numpy()

In [ ]:
def main():
    # Tham số
    BATCH_SIZE = 64
    EPOCHS = 10
    LEARNING_RATE = 0.001
    VECTOR_SIZE = 300
    WINDOW_SIZE = 5
    
    # Đường dẫn files
    DATA_PATH = "sample_data.csv"
    PRETRAINED_PATH = "word2vec_pretrained.bin"  # phải trùng với tên file đã tải ở cell trước
    
    # Kiểm tra files tồn tại
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError(f"Không tìm thấy file dữ liệu: {DATA_PATH}")
    if not os.path.exists(PRETRAINED_PATH):
        raise FileNotFoundError(f"Không tìm thấy pretrained model: {PRETRAINED_PATH}")
    
    print("Đang load dữ liệu...")
    # Load dữ liệu
    train_texts, val_texts = load_and_preprocess_data(DATA_PATH)
    
    print("Khởi tạo datasets...")
    # Khởi tạo datasets
    train_text_dataset = TextDataset(train_texts)
    val_text_dataset = TextDataset(val_texts)
    
    train_dataset = Doc2VecDataset(train_text_dataset, window_size=WINDOW_SIZE)
    val_dataset = Doc2VecDataset(val_text_dataset, window_size=WINDOW_SIZE)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    print("Khởi tạo model...")
    # Khởi tạo model
    model = Doc2VecWithPretrained(
        pretrained_path=PRETRAINED_PATH,
        vocab_size=train_text_dataset.vocab_size,
        doc_count=len(train_texts),
        vector_size=VECTOR_SIZE
    )
    
    print("Bắt đầu huấn luyện...")
    # Train model
    train_doc2vec(model, train_loader, val_loader, EPOCHS, LEARNING_RATE)
    print("Huấn luyện hoàn tất!")

if __name__ == "__main__":
    main()